In [1]:
import requests
import csv
from bs4 import BeautifulSoup as bs
from random import randint
from time import sleep
from urllib.parse import urljoin
import os
import snowflake.connector
from datetime import datetime
import uuid
import re
import pandas as pd
import time
from word2number import w2n  # Library to convert numbers to words

In [2]:
URL = 'https://www.espncricinfo.com/records/trophy/indian-premier-league-117'
sleep_delay = 5
#https://rforotb-ka22072.snowflakecomputing.com
# Snowflake connection parameters
snowflake_account = 'rforotb-ka22072'
snowflake_warehouse= 'COMPUTE_WH'
snowflake_database = 'MYCRICKET'
snowflake_schema = 'DATA1'
snowflake_user = 'comicsvibe'
snowflake_password = 'Comicsvibe123!'
snowflake_stage = 'MYSTAGE1'

# Establish Snowflake connection
conn = snowflake.connector.connect(
    account=snowflake_account,
    warehouse=snowflake_warehouse,
    database=snowflake_database,
    schema=snowflake_schema,
    user=snowflake_user,
    password=snowflake_password,
    stage=snowflake_stage
)

In [3]:
cursor = conn.cursor()

# Function to create a table in Snowflake based on the CSV file
def create_table_from_csv(conn, csv_file):
    table_name = csv_file.split("/")[-1].split(".")[0]
    print(table_name)

    # Manually set data types for LAST_UPDATE and all other columns as 'object'
    data_types = {'LAST_UPDATE': 'object'}
    df = pd.read_csv(csv_file, nrows=1, dtype=data_types)

    # Map data types based on column values
    data_types = {
        'int64': 'NUMBER',
        'float64': 'NUMBER',
        'object': 'VARCHAR(255)',
    }

    create_table_sql = f"CREATE OR REPLACE TABLE {snowflake_database}.{snowflake_schema}.{table_name} ("

    column_counts = {}  # Dictionary to store column name occurrences

    for col_name, col_type in zip(df.columns, df.dtypes):
        if col_name == 'LAST_UPDATE':
            data_type = 'TIMESTAMP_NTZ(9)'  # For 'LAST_UPDATE' column
        else:
            # Check if the column contains only NULL values
            if df[col_name].isnull().all():
                data_type = 'NUMBER'  # Treat NULL columns as 'NUMBER' data type
            else:
                data_type = data_types.get(str(col_type), 'VARCHAR')  # Default to VARCHAR if type not found

        # Handle 'Null' values in column names
        #col_name = col_name.replace('Null', '_null_')
        
        # Replace spaces in column names with underscores
        col_name = col_name.replace(' ', '_')
        
        # Handle column names starting with a number by adding 'S_' in front
        if col_name[0].isdigit():
            col_name = 'S_' + col_name
        
        # Handle duplicate column names by appending "_1", "_2", etc.
        if col_name in column_counts:
            column_counts[col_name] += 1
            col_name = f"{col_name}_{column_counts[col_name]}"
        else:
            column_counts[col_name] = 1

        create_table_sql += f"{col_name} {data_type}, "

    create_table_sql = create_table_sql.rstrip(', ') + ")"
    print(create_table_sql)

    try:
        cursor = conn.cursor()
        cursor.execute(create_table_sql)
        print(f"Table '{table_name}' created successfully.")
    except snowflake.connector.errors.ProgrammingError as e:
        print(f"Error creating table '{table_name}': {e}")
    finally:
        cursor.close()

# Use the Snowflake Python Connector to query the stage and get the list of files
sql = f"LIST @{snowflake_stage}"
cursor.execute(sql)
csv_files = [row[0] for row in cursor.fetchall()]
print(csv_files)

# Create tables based on CSV files with a 5-second delay between each table creation
for csv_file in csv_files:
    csv_file_name = csv_file.split("/")[-2]
    create_table_from_csv(conn, csv_file_name)
    time.sleep(5)  # Add a 5-second delay between table creations

# Close Snowflake connection
conn.close()

['mystage3/abandoned_matches_283285.csv/abandoned_matches_283285.csv.gz', 'mystage3/best_career_bowling_average_283258.csv/best_career_bowling_average_283258.csv.gz', 'mystage3/best_career_bowling_average_without_qualification_283899.csv/best_career_bowling_average_without_qualification_283899.csv.gz', 'mystage3/best_career_economy_rate_283267.csv/best_career_economy_rate_283267.csv.gz', 'mystage3/best_career_strike_rate_283276.csv/best_career_strike_rate_283276.csv.gz', 'mystage3/best_economy_rate_in_an_innings_283890.csv/best_economy_rate_in_an_innings_283890.csv.gz', 'mystage3/best_figures_in_a_innings_by_a_captain_283935.csv/best_figures_in_a_innings_by_a_captain_283935.csv.gz', 'mystage3/best_figures_in_a_innings_on_debut_283833.csv/best_figures_in_a_innings_on_debut_283833.csv.gz', 'mystage3/best_figures_in_a_innings_when_on_the_losing_side_283926.csv/best_figures_in_a_innings_when_on_the_losing_side_283926.csv.gz', 'mystage3/best_figures_in_an_innings_283973.csv/best_figures_in_

Table 'abandoned_matches_283285' created successfully.
best_career_bowling_average_283258
CREATE OR REPLACE TABLE MYCRICKET.DATA3.best_career_bowling_average_283258 (uuid VARCHAR(255), LAST_UPDATE TIMESTAMP_NTZ(9), Player VARCHAR(255), Span NUMBER, Mat NUMBER, Overs NUMBER, Mdns NUMBER, Balls NUMBER, Runs NUMBER, Wkts NUMBER, BBI NUMBER, BBM NUMBER, Ave NUMBER, Econ NUMBER, SR NUMBER, S_4 NUMBER, S_5 NUMBER, S_10 VARCHAR(255))
Table 'best_career_bowling_average_283258' created successfully.
best_career_bowling_average_without_qualification_283899
CREATE OR REPLACE TABLE MYCRICKET.DATA3.best_career_bowling_average_without_qualification_283899 (uuid VARCHAR(255), LAST_UPDATE TIMESTAMP_NTZ(9), Player VARCHAR(255), Span NUMBER, Mat NUMBER, Balls NUMBER, Overs NUMBER, Mdns VARCHAR(255), Runs NUMBER, Wkts NUMBER, BBI NUMBER, BBM NUMBER, Ave NUMBER, Econ NUMBER, SR NUMBER, S_4 VARCHAR(255), S_5 VARCHAR(255), S_10 VARCHAR(255))
Table 'best_career_bowling_average_without_qualification_283899' c

fastest_to_100_wickets_1286053
CREATE OR REPLACE TABLE MYCRICKET.DATA3.fastest_to_100_wickets_1286053 (uuid VARCHAR(255), LAST_UPDATE TIMESTAMP_NTZ(9), Player VARCHAR(255), Opposition VARCHAR(255), Ground VARCHAR(255), Match_Date NUMBER, Debut NUMBER, Time NUMBER, Mat NUMBER, Scorecard VARCHAR(255))
Table 'fastest_to_100_wickets_1286053' created successfully.
fastest_to_2000_runs_1213211
CREATE OR REPLACE TABLE MYCRICKET.DATA3.fastest_to_2000_runs_1213211 (uuid VARCHAR(255), LAST_UPDATE TIMESTAMP_NTZ(9), Player VARCHAR(255), Opposition VARCHAR(255), Ground VARCHAR(255), Match_Date NUMBER, Debut NUMBER, Time NUMBER, Mat NUMBER, Inns NUMBER, Scorecard VARCHAR(255))
Table 'fastest_to_2000_runs_1213211' created successfully.
fastest_to_2500_runs_1288793
CREATE OR REPLACE TABLE MYCRICKET.DATA3.fastest_to_2500_runs_1288793 (uuid VARCHAR(255), LAST_UPDATE TIMESTAMP_NTZ(9), Player VARCHAR(255), Opposition VARCHAR(255), Ground VARCHAR(255), Match_Date NUMBER, Debut NUMBER, Time NUMBER, Mat NUMB

Table 'highest_partnership_for_the_first_wicket_283608' created successfully.
highest_partnership_for_the_fourth_wicket_283587
CREATE OR REPLACE TABLE MYCRICKET.DATA3.highest_partnership_for_the_fourth_wicket_283587 (uuid VARCHAR(255), LAST_UPDATE TIMESTAMP_NTZ(9), Partners VARCHAR(255), Runs NUMBER, Team VARCHAR(255), Opposition VARCHAR(255), Ground VARCHAR(255), Match_Date NUMBER, Scorecard VARCHAR(255))
Table 'highest_partnership_for_the_fourth_wicket_283587' created successfully.
highest_partnership_for_the_ninth_wicket_283551
CREATE OR REPLACE TABLE MYCRICKET.DATA3.highest_partnership_for_the_ninth_wicket_283551 (uuid VARCHAR(255), LAST_UPDATE TIMESTAMP_NTZ(9), Partners VARCHAR(255), Runs NUMBER, Team VARCHAR(255), Opposition VARCHAR(255), Ground VARCHAR(255), Match_Date NUMBER, Scorecard VARCHAR(255))
Table 'highest_partnership_for_the_ninth_wicket_283551' created successfully.
highest_partnership_for_the_second_wicket_283601
CREATE OR REPLACE TABLE MYCRICKET.DATA3.highest_partne

Table 'list_of_series_results_335433' created successfully.
longest_careers_1219896
CREATE OR REPLACE TABLE MYCRICKET.DATA3.longest_careers_1219896 (uuid VARCHAR(255), LAST_UPDATE TIMESTAMP_NTZ(9), Player VARCHAR(255), From NUMBER, To NUMBER, Span NUMBER, Mat NUMBER)
Error creating table 'longest_careers_1219896': 001003 (42000): SQL compilation error:
syntax error line 1 at position 135 unexpected 'From'.
syntax error line 1 at position 148 unexpected 'To'.
longest_individual_innings_by_balls_283131
CREATE OR REPLACE TABLE MYCRICKET.DATA3.longest_individual_innings_by_balls_283131 (uuid VARCHAR(255), LAST_UPDATE TIMESTAMP_NTZ(9), Player VARCHAR(255), Runs NUMBER, Mins VARCHAR(255), Balls NUMBER, S_4s NUMBER, S_6s NUMBER, SR NUMBER, Team VARCHAR(255), Opposition VARCHAR(255), Ground VARCHAR(255), Match_Date NUMBER, Scorecard VARCHAR(255))
Table 'longest_individual_innings_by_balls_283131' created successfully.
longest_intervals_between_appearances_1222956
CREATE OR REPLACE TABLE MYCRIC

most_bowlers_in_a_match_282921
CREATE OR REPLACE TABLE MYCRICKET.DATA3.most_bowlers_in_a_match_282921 (uuid VARCHAR(255), LAST_UPDATE TIMESTAMP_NTZ(9), Team_1 VARCHAR(255), Team_2 VARCHAR(255), Num NUMBER, Winner VARCHAR(255), Ground VARCHAR(255), Match_Date NUMBER, Scorecard VARCHAR(255))
Table 'most_bowlers_in_a_match_282921' created successfully.
most_bowlers_in_an_innings_282928
CREATE OR REPLACE TABLE MYCRICKET.DATA3.most_bowlers_in_an_innings_282928 (uuid VARCHAR(255), LAST_UPDATE TIMESTAMP_NTZ(9), Team VARCHAR(255), Num NUMBER, Inns NUMBER, _1 VARCHAR(255), Result VARCHAR(255), Opposition NUMBER, Score NUMBER, Overs VARCHAR(255), Ground NUMBER, Match_Date VARCHAR(255), Scorecard NUMBER)
Table 'most_bowlers_in_an_innings_282928' created successfully.
most_bowlers_taking_a_wicket_in_an_innings_282913
CREATE OR REPLACE TABLE MYCRICKET.DATA3.most_bowlers_taking_a_wicket_in_an_innings_282913 (uuid VARCHAR(255), LAST_UPDATE TIMESTAMP_NTZ(9), Team VARCHAR(255), Num NUMBER, Tot NUMBER, 

most_consecutive_matches_without_victory_283096
CREATE OR REPLACE TABLE MYCRICKET.DATA3.most_consecutive_matches_without_victory_283096 (uuid VARCHAR(255), LAST_UPDATE TIMESTAMP_NTZ(9), Team VARCHAR(255), Mat NUMBER, Result VARCHAR(255), Margin NUMBER, Opposition VARCHAR(255), Ground VARCHAR(255), Match_Date NUMBER, Scorecard VARCHAR(255))
Table 'most_consecutive_matches_without_victory_283096' created successfully.
most_consecutive_wins_283111
CREATE OR REPLACE TABLE MYCRICKET.DATA3.most_consecutive_wins_283111 (uuid VARCHAR(255), LAST_UPDATE TIMESTAMP_NTZ(9), Team VARCHAR(255), Mat NUMBER, Margin NUMBER, Opposition VARCHAR(255), Ground VARCHAR(255), Match_Date NUMBER, Scorecard VARCHAR(255))
Table 'most_consecutive_wins_283111' created successfully.
most_dismissals_in_an_innings_283377
CREATE OR REPLACE TABLE MYCRICKET.DATA3.most_dismissals_in_an_innings_283377 (uuid VARCHAR(255), LAST_UPDATE TIMESTAMP_NTZ(9), Player VARCHAR(255), Dis NUMBER, Ct NUMBER, St NUMBER, Inns NUMBER, Team V

Table 'most_maidens_in_an_innings_311985' created successfully.
most_maidens_in_career_311984
CREATE OR REPLACE TABLE MYCRICKET.DATA3.most_maidens_in_career_311984 (uuid VARCHAR(255), LAST_UPDATE TIMESTAMP_NTZ(9), Player VARCHAR(255), Span NUMBER, Mat NUMBER, Inns NUMBER, Balls NUMBER, Overs NUMBER, Mdns NUMBER, Runs NUMBER, Wkts NUMBER, BBI NUMBER, BBM NUMBER, Ave NUMBER, Econ NUMBER, SR NUMBER, S_4 VARCHAR(255), S_5 VARCHAR(255), S_10 VARCHAR(255))
Table 'most_maidens_in_career_311984' created successfully.
most_matches_as_a_match_referee_283386
CREATE OR REPLACE TABLE MYCRICKET.DATA3.most_matches_as_a_match_referee_283386 (uuid VARCHAR(255), LAST_UPDATE TIMESTAMP_NTZ(9), Player VARCHAR(255), Span NUMBER, Matches NUMBER)
Table 'most_matches_as_a_match_referee_283386' created successfully.
most_matches_as_an_umpire_283392
CREATE OR REPLACE TABLE MYCRICKET.DATA3.most_matches_as_an_umpire_283392 (uuid VARCHAR(255), LAST_UPDATE TIMESTAMP_NTZ(9), Umpire VARCHAR(255), Span NUMBER, Matches 

most_runs_in_an_innings_by_a_wicketkeeper_284209
CREATE OR REPLACE TABLE MYCRICKET.DATA3.most_runs_in_an_innings_by_a_wicketkeeper_284209 (uuid VARCHAR(255), LAST_UPDATE TIMESTAMP_NTZ(9), Player VARCHAR(255), Runs NUMBER, Mins NUMBER, Balls NUMBER, S_4s NUMBER, S_6s NUMBER, SR NUMBER, Team VARCHAR(255), Opposition VARCHAR(255), Ground VARCHAR(255), Match_Date NUMBER, Scorecard VARCHAR(255))
Table 'most_runs_in_an_innings_by_a_wicketkeeper_284209' created successfully.
most_runs_in_an_innings_by_batting_position_284241
CREATE OR REPLACE TABLE MYCRICKET.DATA3.most_runs_in_an_innings_by_batting_position_284241 (uuid VARCHAR(255), LAST_UPDATE TIMESTAMP_NTZ(9), Player VARCHAR(255), Runs NUMBER, Mins VARCHAR(255), Balls NUMBER, S_4s NUMBER, S_6s NUMBER, SR NUMBER, Team VARCHAR(255), Opposition VARCHAR(255), Ground VARCHAR(255), Match_Date NUMBER, Scorecard VARCHAR(255))
Table 'most_runs_in_an_innings_by_batting_position_284241' created successfully.
most_runs_in_an_innings_progressive_record

FileNotFoundError: [Errno 2] No such file or directory: 'most_wickets_in_a_calendar_year_283961.csv'